In [1]:
import pandas as pd
import plotly.graph_objects as go

# Load the CSV file into a pandas DataFrame
try:
    dataF = pd.read_csv("SOLUSDT.csv")
except FileNotFoundError:
    print("The file 'SOLUSDT.csv' was not found.")
    raise


c:\Users\Matilde Sierra\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [4]:
dataF.columns = dataF.columns.str.lower()
dataF = dataF[dataF["high"] != dataF['low']]
dataF.reset_index(drop=True, inplace=True)
dataF

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore
0,2020-08-11,2.8500,3.5208,2.8433,3.2985,1552384.780,1.597190e+12,4.939149e+06,13490.0,741770.790,2.370193e+06,0.0
1,2020-08-12,3.2985,3.9289,3.0800,3.7558,1737042.950,1.597277e+12,6.176154e+06,21118.0,889133.500,3.161944e+06,0.0
2,2020-08-13,3.7500,4.1387,3.5003,3.7300,1685759.240,1.597363e+12,6.446568e+06,22922.0,716358.460,2.755765e+06,0.0
3,2020-08-14,3.7207,3.7676,3.3210,3.4099,1474161.790,1.597450e+12,5.205834e+06,18581.0,578651.840,2.048897e+06,0.0
4,2020-08-15,3.4181,3.7400,3.1500,3.1730,1070233.200,1.597536e+12,3.658006e+06,13903.0,439623.970,1.510473e+06,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1382,2024-05-24,176.1500,176.1500,162.5000,167.7000,4819660.642,1.716595e+12,8.100286e+08,856617.0,2592690.356,4.361727e+08,0.0
1383,2024-05-25,167.7000,170.5500,165.9000,168.3500,1911443.135,1.716682e+12,3.216913e+08,405433.0,913344.720,1.537293e+08,0.0
1384,2024-05-26,168.3500,168.4000,161.0000,163.4100,3126117.989,1.716768e+12,5.128948e+08,537560.0,1562641.600,2.563656e+08,0.0
1385,2024-05-27,163.4200,172.2800,163.2600,170.1500,3260017.055,1.716854e+12,5.473441e+08,591480.0,1688224.025,2.833042e+08,0.0


In [5]:
# Define the rejection signal function
def rejection_signal(row):
    body = abs(row['open'] - row['close'])
    upper_wick = row['high'] - max(row['open'], row['close'])
    lower_wick = min(row['open'], row['close']) - row['low']

    # Bullish signal
    if row['open'] < row['close'] and upper_wick < body / 10 and lower_wick > body * 5:
        return 2
    # Bearish signal
    elif row['open'] > row['close'] and upper_wick > body * 5 and lower_wick < body / 10:
        return 1
    # No signal
    else:
        return 0

# Define the engulfing signal function
def engulfing_signal(previous_candle, current_candle):
    # Bullish engulfing
    if (current_candle['close'] > previous_candle['open'] and
            current_candle['open'] < previous_candle['close'] and
            previous_candle['open'] > previous_candle['close']):
        return 2
    # Bearish engulfing
    elif (current_candle['open'] > previous_candle['close'] and
          current_candle['close'] < previous_candle['open'] and
          previous_candle['close'] > previous_candle['open']):
        return 1
    # No signal
    else:
        return 0

# Generate rejection signals
dataF['rejection_signal'] = dataF.apply(rejection_signal, axis=1)

# Generate engulfing signals
engulfing_signals = [0] * len(dataF)
for i in range(1, len(dataF)):
    engulfing_signals[i] = engulfing_signal(dataF.iloc[i-1], dataF.iloc[i])
dataF['engulfing_signal'] = engulfing_signals

In [6]:
# Analyze the engulfing signals
up_count = 0
down_count = 0
total_count = 0

for i in range(len(dataF) - 1):
    if dataF.engulfing_signal.iloc[i] == 1:
        total_count += 1
        if dataF.close.iloc[i+1] > dataF.open.iloc[i+1]:
            up_count += 1
        elif dataF.close.iloc[i+1] < dataF.open.iloc[i+1]:
            down_count += 1

if total_count > 0:
    up_percentage = (up_count / total_count) * 100
    down_percentage = (down_count / total_count) * 100
    print(f"Up Percentage: {up_percentage:.2f}%, Down Percentage: {down_percentage:.2f}%, Total Count: {total_count}")
else:
    print("No engulfing signals found.")

Up Percentage: 51.11%, Down Percentage: 48.89%, Total Count: 45


In [7]:
dataF[dataF["engulfing_signal"]==1]

,open_time,open,high,low,close,volume,close_time,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume,ignore,rejection_signal,engulfing_signal
14,2020-08-25,3.3406,3.3654,3.0001,3.2500,1.149187e+06,1.598400e+12,3.629891e+06,14226.0,477028.870,1.506233e+06,0.0,0,1
74,2020-10-24,1.9652,1.9730,1.8800,1.9142,7.480938e+05,1.603584e+12,1.442961e+06,7136.0,330710.090,6.387923e+05,0.0,0,1
92,2020-11-11,2.2665,2.3000,1.9673,2.0000,1.863686e+06,1.605139e+12,3.969960e+06,15371.0,815224.570,1.739081e+06,0.0,0,1
137,2020-12-26,1.4488,1.4497,1.3118,1.3258,9.121923e+05,1.609027e+12,1.249353e+06,14371.0,432672.840,5.931746e+05,0.0,0,1
147,2021-01-05,2.4911,2.5093,2.0621,2.1636,6.702007e+06,1.609891e+12,1.513356e+07,102299.0,3156689.790,7.157811e+06,0.0,0,1
163,2021-01-21,3.7199,3.7411,2.9717,2.9899,4.385839e+06,1.611274e+12,1.473715e+07,60540.0,1969558.820,6.631085e+06,0.0,0,1
169,2021-01-27,4.0418,4.0422,3.5114,3.6750,2.491758e+06,1.611792e+12,9.309019e+06,38794.0,1151705.710,4.306099e+06,0.0,0,1
189,2021-02-16,8.8660,9.0000,7.9601,8.3002,2.257218e+06,1.613520e+12,1.915301e+07,49157.0,1033564.660,8.772332e+06,0.0,0,1
198,2021-02-25,17.1745,17.6900,13.6579,13.8442,9.520374e+06,1.614298e+12,1.512759e+08,210627.0,4851015.200,7.719722e+07,0.0,0,1
205,2021-03-04,14.0936,14.2737,12.7360,13.0508,2.764372e+06,1.614902e+12,3.714358e+07,75817.0,1365819.490,1.833324e+07,0.0,0,1


In [8]:
# Visualize the candlestick chart
st = 0
dfpl = dataF[st:st+150].copy()
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                                     open=dfpl['open'],
                                     high=dfpl['high'],
                                     low=dfpl['low'],
                                     close=dfpl['close'])])

fig.show()